In [2]:
from pywgraph import *

In [3]:
grafo = WeightedDirectedGraph.from_dict(
    {
        "A": {"B": 1, "F": 1},
        "B": {"C": 1, "A":1},
        "C": {"A":1, "D":1},
        "D": {"A":1, "E":1},
        "E": {"B":1, "F": 1},
        "F":{"E":1, "A":1}
    }
)

In [111]:
class MegaExplorer:

    def __init__(self, path: list[str], visitations: dict[str, int] = {}) -> None: 
        self._path = path
        self._visitations = visitations

    @property
    def path(self) -> list[str]:
        return self._path
    
    @property
    def visitations(self) -> dict[str, int]:
        return self._visitations
    
    def __hash__(self) -> int:
        hash_list = hash(tuple(self.path))
        hash_dict = hash(tuple(sorted(list(self.visitations.items()), key=lambda x: x[0])))
        return hash_list ^ hash_dict
    
    def __eq__(self, other: object) -> bool:
        if isinstance(other, MegaExplorer):
            return (self.path, self.visitations) == (other.path, other.visitations)
        return False

    def __len__(self) -> int:
        return len(self.path)
    
    def __repr__(self) -> str:
        return f"PathExplorer({self.path}, {self.visitations})"


In [108]:
def iter_aux(
    grafo: WeightedDirectedGraph,
    explorer: MegaExplorer,
    target: str,
    general_max_visitations: int,
    specific_max_visitations: dict[str, int],
) -> tuple[list, str] | tuple[MegaExplorer, str]:
    current_node = explorer.path[-1]
    visitations = explorer.visitations.copy()
    visitations[current_node] = visitations.get(current_node, 0) + 1
    current_node_vistiations = visitations[current_node]
    current_node_max_vistiations = specific_max_visitations.get(
        current_node, general_max_visitations
    )

    if (current_node == target) and (
        current_node_vistiations == current_node_max_vistiations
    ):
        return explorer.path, "path"

    if current_node_vistiations > current_node_max_vistiations:
        return [], "dead explorer"

    children = grafo.children(current_node)
    forbidden_nodes = {
        node
        for node, rep in visitations.items()
        if rep >= specific_max_visitations.get(node, general_max_visitations)
    }
    unexplored_nodes = children - forbidden_nodes
    if not unexplored_nodes:
        return [], "dead explorer"

    new_explorers = [
        MegaExplorer(explorer.path + [node], visitations) for node in unexplored_nodes
    ]
    return new_explorers, "continue"

In [119]:
def mega_find_paths(
    grafo: WeightedDirectedGraph,
    start: str,
    end: str,
    general_max_visitations: int = 1,
    specific_max_visitations: dict[str, int] = {},
) -> list[list[str]]:

    explorers: list[MegaExplorer] = [MegaExplorer([start])]
    all_paths: list[list[str]] = []

    it = 0
    while explorers and it <= 200:
        it += 1
        result, state = iter_aux(
            grafo,
            explorers.pop(0),
            end,
            general_max_visitations,
            specific_max_visitations,
        )
        if state == "path":
            all_paths.append(result)
        elif state == "continue":
            new_explorers = list(set(result) - set(explorers))
            explorers.extend(new_explorers)

    return all_paths

In [121]:
mega_find_paths(grafo, "A", "B")

[['A', 'B'], ['A', 'F', 'E', 'B']]

In [115]:
sorted(list(grafo.get_node_cycles("A")), key=len)

[Cycle(['A', 'F']),
 Cycle(['A', 'B']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B']),
 Cycle(['A', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D'])]

In [5]:
class PathExplorer(list[str]):

    def __init__(self, path: list[str] = [], visited: set[str] = set()):
        self._path = path
        self._visited = visited

    @property
    def path(self) -> list[str]:
        return self._path
    
    @property
    def visited(self) -> set[str]:
        return self._visited
    
    def __hash__(self) -> int:
        return hash((tuple(self.path), tuple(self.visited)))
    
    def __eq__(self, other: object) -> bool:
        if isinstance(other, PathExplorer):
            return (self.path, self.visited) == (other.path, other.visited)
        return False
    
    def __repr__(self) -> str:
        return f"PathExplorer({self._path}, {self._visited})"
    
    def __len__(self) -> int:
        return len(self._path)

In [6]:
def _cycle_aux(
    grafo: WeightedDirectedGraph, explorer: PathExplorer, target: str
) -> tuple[list, str] | tuple[Cycle, str]:
    current_node = explorer.path[-1]
    if current_node == target:
        return Cycle(explorer.path[:-1]), "cycle"
    
    children = grafo.children(current_node)
    unexplored_nodes = children - explorer.visited
    if not unexplored_nodes:
        return [], "dead explorer"
    
    updated_visited = explorer.visited | {current_node}
    new_explorers = [
        PathExplorer(explorer.path + [node], updated_visited)
        for node in unexplored_nodes
    ]
    return new_explorers, "continue"

In [7]:
def _get_node_cycles(grafo: WeightedDirectedGraph, start: str) -> list[list[str]]:
    first_children = grafo.children(start)
    if not first_children:
        return []

    explorers: list[PathExplorer] = [
        PathExplorer([start, child]) for child in first_children
    ]
    cycles: list[Cycle] = []

    while explorers: 
        result, state = _cycle_aux(grafo, explorers.pop(0), start)                              
        if state == "cycle":
            cycles.append(result)
        elif state == "continue":
            new_explorers = list(set(result) - set(explorers))
            explorers.extend(new_explorers)

    return cycles

In [22]:
def _path_aux(
    grafo: WeightedDirectedGraph, explorer: PathExplorer, target: str
) -> tuple[list, str] | tuple[Cycle, str]:
    current_node = explorer.path[-1]
    if current_node == target:
        return explorer.path, "path"
    
    children = grafo.children(current_node)
    unexplored_nodes = children - explorer.visited
    if not unexplored_nodes:
        return [], "dead explorer"
    
    updated_visited = explorer.visited | {current_node}
    new_explorers = [
        PathExplorer(explorer.path + [node], updated_visited)
        for node in unexplored_nodes
    ]
    return new_explorers, "continue"

In [31]:
def _find_all_paths(grafo: WeightedDirectedGraph, start: str, end: str) -> list[list[str]]:
    first_children = grafo.children(start)
    if not first_children:
        return []

    explorers: list[PathExplorer] = [
        PathExplorer([start, child]) for child in first_children
    ]
    all_paths: list[list[str]] = []

    while explorers:
        result, state = _path_aux(grafo, explorers.pop(0), end)                  
        if state == "path":
            all_paths.append(result)
        elif state == "continue":
            new_explorers = list(set(result) - set(explorers))
            explorers.extend(new_explorers)

    return all_paths

In [10]:
grafo.get_node_cycles("A")

{Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'B', 'C', 'D']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'B']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B']),
 Cycle(['A', 'F'])}

In [33]:
_find_all_paths(grafo, "A", "F")

[['A', 'F'],
 ['A', 'B', 'A', 'F'],
 ['A', 'B', 'C', 'A', 'F'],
 ['A', 'B', 'C', 'D', 'E', 'F'],
 ['A', 'B', 'C', 'D', 'A', 'F']]

In [7]:
grafo.get_node_cycles("B")

{Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'B', 'C', 'D']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'B']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B']),
 Cycle(['B', 'C', 'D', 'E'])}

In [8]:
grafo.get_node_cycles("C")

{Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'B', 'C', 'D']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['B', 'C', 'D', 'E'])}

In [9]:
grafo.get_node_cycles("D")

{Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D']),
 Cycle(['B', 'C', 'D', 'E'])}

In [10]:
grafo.get_node_cycles("E")

{Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B']),
 Cycle(['B', 'C', 'D', 'E']),
 Cycle(['E', 'F'])}

In [11]:
grafo.get_node_cycles("F")

{Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B']),
 Cycle(['A', 'F']),
 Cycle(['E', 'F'])}

In [12]:
grafo.cycles

{Cycle(['A', 'B', 'C', 'D', 'E', 'F']),
 Cycle(['A', 'B', 'C', 'D']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'B']),
 Cycle(['A', 'F', 'E', 'B', 'C', 'D']),
 Cycle(['A', 'F', 'E', 'B', 'C']),
 Cycle(['A', 'F', 'E', 'B']),
 Cycle(['A', 'F']),
 Cycle(['B', 'C', 'D', 'E']),
 Cycle(['E', 'F'])}

In [13]:
grafo.is_conmutative

True

In [14]:
import numpy as np

def vector_group_multiplication() -> Group:
    group = Group(
        name="Vectors of dimension 2 with multiplication",
        identity=np.ones(2),
        operation=lambda x, y: x * y,
        inverse_function=lambda x: 1 / x,
        hash_function=lambda x: hash(tuple(x)),
    )
    return group


def vector_group_addition() -> Group:
    group = Group(
        name="Vectors of dimension 2 with addition",
        identity=np.zeros(2),
        operation=lambda x, y: x + y,
        inverse_function=lambda x: -x,
        hash_function=lambda x: hash(tuple(x)),
    )
    return group


_array_dict_graph: dict[str, dict[str, Any]] = {
    "A": {
        "B": np.array([1.0, 2.0]),
        "C": np.array([3.0, 4.0]),
    },
    "B": {
        "C": np.array([-5.0, 1.3]),
        "D": np.array([2.0, 1.0]),
    },
    "C": {
        "D": np.array([-1.0, 1.0]),
    },
    "D": {},
    "Z": {},
}


def multiplication_graph() -> WeightedDirectedGraph:
    graph = WeightedDirectedGraph.from_dict(
        _array_dict_graph, vector_group_multiplication()
    )
    return graph


def addition_graph() -> WeightedDirectedGraph:
    graph = WeightedDirectedGraph.from_dict(_array_dict_graph, vector_group_addition())
    return graph

In [25]:
addition_graph().add_reverse_edges().is_conmutative

False

In [26]:
complete_graph = addition_graph().add_reverse_edges()
group = complete_graph.group
identity = group.identity
bad = [
    cycle
    for cycle in complete_graph.cycles
    if not group.equal(complete_graph.path_weight(cycle), identity)
]
bad 

[Cycle(['B', 'C', 'D']),
 Cycle(['B', 'D', 'C']),
 Cycle(['A', 'C', 'D', 'B']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'C', 'B']),
 Cycle(['A', 'B', 'D', 'C'])]

In [27]:
[
    complete_graph.path_weight(cycle)
    for cycle in bad
]

[array([-8. ,  1.3]),
 array([ 8. , -1.3]),
 array([-1.,  2.]),
 array([-7. , -0.7]),
 array([7. , 0.7]),
 array([ 1., -2.])]

In [24]:
complete_graph

Nodes: {'B', 'C', 'Z', 'D', 'A'}
Edges:
A -> C: [3. 4.]
A -> B: [1. 2.]
D -> C: [-1.  1.]
B -> A: [1.  0.5]
C -> D: [-1.  1.]
C -> A: [0.33333333 0.25      ]
B -> C: [-5.   1.3]
B -> D: [2. 1.]
C -> B: [-0.2         0.76923077]
D -> B: [0.5 1. ]

In [18]:
todos = set()
for node in complete_graph.nodes: 
    todos.update(complete_graph.get_node_cycles(node))

todos

{Cycle(['A', 'B', 'C']),
 Cycle(['A', 'B', 'D', 'C']),
 Cycle(['A', 'B']),
 Cycle(['A', 'C', 'B']),
 Cycle(['A', 'C', 'D', 'B']),
 Cycle(['A', 'C']),
 Cycle(['B', 'C', 'D']),
 Cycle(['B', 'C']),
 Cycle(['B', 'D', 'C']),
 Cycle(['B', 'D']),
 Cycle(['C', 'D'])}

In [19]:
[cycle for cycle in complete_graph.cycles]

[Cycle(['B', 'C', 'D']),
 Cycle(['C', 'D']),
 Cycle(['B', 'D', 'C']),
 Cycle(['A', 'C', 'D', 'B']),
 Cycle(['A', 'C']),
 Cycle(['A', 'B']),
 Cycle(['B', 'C']),
 Cycle(['A', 'B', 'C']),
 Cycle(['A', 'C', 'B']),
 Cycle(['B', 'D']),
 Cycle(['A', 'B', 'D', 'C'])]

In [32]:
grafo = WeightedDirectedGraph.from_dict(
    {
        "A": {"B": np.array([1, 2.0]), "C": np.array([3.0, 4.0])},
        "B": {"C": np.array([2.0, 2.0])},
        "C": {},
    },
    vector_group_addition(),
)

In [36]:
grafo.add_reverse_edges().cycles

{Cycle(['A', 'B', 'C']),
 Cycle(['A', 'B']),
 Cycle(['A', 'C', 'B']),
 Cycle(['A', 'C']),
 Cycle(['B', 'C'])}